In [1]:
from os.path import join, basename, splitext
import json
from glob import glob
from tqdm import tqdm
from itertools import islice
import numpy as np

from common import tokenize
from nltk.tokenize import sent_tokenize
import logging

from IPython.display import Markdown, display, HTML
from common import get_all_docs, word_tokenizer

logger = logging.getLogger(__name__)
DATA_FOLDER = '../data/'

Using TensorFlow backend.


In [2]:
all_docs = get_all_docs(DATA_FOLDER)
val_docs = glob(join(DATA_FOLDER, 'validate/*.txt'))

with open(join(DATA_FOLDER, 'best.json'), 'r') as f:
    best = json.load(f)
    
with open(join(DATA_FOLDER, 'gold.json'), 'r') as f:
    gold = json.load(f)

In [3]:
def hprint(string, i=1):
    display(HTML('<h%s>%s</h%s>' % (i, string, i)))

def print_file(sub_name, domain, i=1, tokenize=False):    
    found = [fn for fn in domain if sub_name in fn]
    for fn in found:
        hprint(fn + '\n', i)
        with open(fn, 'r') as f:
            text = f.read()
            if tokenize:
                print(tokenize(text))
            else:
                print(text)
#         !cat {'\''+fn+'\''}

def inspect_result(qname, start_index=0, end_index=2):
    true_docs = gold[qname]

    hprint('Query\n')
    print_file(qname, val_docs, 5)
    for i, tname in enumerate(true_docs):
        hprint('True %s\n' %(i+1))
        print_file(tname, all_docs, 5)

    for i, bname in enumerate(best[qname][start_index:end_index]):
        hprint('Found %s\n' % (i+1))
        print_file(bname, all_docs, 5)    

In [4]:
val_docs_ids = set([splitext(basename(f))[0] for f in val_docs])
validation_ids = [k for k in gold.keys() if k in val_docs_ids]
print('validation set size %s' % len(validation_ids))
for qname in validation_ids:
    print(qname)

validation set size 130
_40_1a_2011126928A10010101RU
_df_5c_2010114200A10010101RU
_ab_68_2006139128A10010101RU
_1b_60_2010122997A10010101RU
_ef_e0_2010102239A10010101RU
_a5_03_2009140669A10010101RU
_87_7b_2010120096A10010101RU
_c3_b8_2009111341A10010101RU
_ff_96_2010138984A10010101RU
_be_ef_2010152978A10010101RU
_59_59_2009131618A10010101RU
_87_6d_2008148713A10010101RU
_0c_89_2009110686A10010101RU
_03_43_2008151996A10010101RU
_94_b1_2009122096A10010101RU
_e6_cc_2009147572A10010101RU
_18_b9_2010101690A10010101RU
_79_fe_2010107007A10010101RU
_ae_fc_2010107173A10010101RU
_8d_51_2010136608A10010101RU
_05_7a_2007148915A10010101RU
_82_1b_2010112743A10010101RU
_91_ca_2010129617A10010101RU
_3f_fb_2010139538A10010101RU
_23_fc_2011105696A10010101RU
_b6_9c_2010104075A10010101RU
_f0_fa_2010100979A10010101RU
_14_9d_2011108876A10010101RU
_a8_27_2010114657A10010101RU
_c9_ad_2010109635A10010101RU
_c0_79_2009148052A10010101RU
_33_40_2009146768A10010101RU
_88_8b_2011124782A10010101RU
_8f_ca_2011124783A1

# Характеристика качества данных

| Заявка | Порочащий д-т без описания   | Хорошее совпадение назаний| Обычный человек отличит|
|------|------|------|
|   _40_1a_2011126928A10010101RU  | 1|||
|   _df_5c_2010114200A10010101RU  | 1|1||
|   _0c_89_2009110686A10010101RU  | |||

# Подход

Так как весь текст патента избыточен для выяснения конкретного новшества данного патента, то имеет смысл брать куски текста вокруг ключевых слов, указывающих на само новшество. Ниже пример таких слов.

Весь текст патента избыточен еще и потому что суть изобретения может быть применима в разных областях, например "кулинария" (подогрев плиты) и "одежда"(подогрев одежды). 

Поиск по ключевым словам:
- из названия патента
- заявлять
- заключать в том , что
- суть
- новый
- применение в качество
- новый технический результат заключаться в возможность
- новый технологичный способ получение
- преимущество
- сущность изобретение
- эффективный метод, средство, способ
- удалось
- задача , на решение который направлять настоящий изобретение
- новизна
- предмет изобретение
- описывать способ получение
- технический результат полезный модель заключаться
- полезность изобретение
    
Можно дополнить этот список наиболее близкими словами к данным, полученных на этом корпусе.
    
Имеет смысл разметить в тексте патентов такие куски, которые указывают на саму суть и обучаться на двух трех предложениях вокруг ключевых слов. Таким образом есть вероятность что мы лучше поймем отличия патентов.

Метод - CNN, которые выделяют наиболее значимые куски текста, делают саммаризацию в режиме unsupervised learning.

Простой способ индикации описания сути изобретения - векторное среднее движущегося n-gramm по тексту. Нужно пройтись так по всему корпусу для каждого предложения, получить векторные представления n-gramm(фраз-индикаторов), взять n ближайших соседий от заранее заданных фраз-индикаторов(по типу тех что приведены выше) - это и будут фразы-индикаторы. При этом можно будет придумывать новые фразы и смотреть, попадают ли они в этот кластер. Также из n-gramm можно будет выкидывать слова, которые не имеют целевой значимости, например:

"РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула".


In [5]:
import gensim

model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/old/w2v_model_300'))
wv = model.wv

In [47]:
wv.most_similar(positive=['результат'], topn=10)

[('получать', 0.666301429271698),
 ('предлагать', 0.6384239792823792),
 ('неудачн', 0.6303017735481262),
 ('заключаться', 0.6248302459716797),
 ('изобретение', 0.597603440284729),
 ('обеспечивать', 0.584223747253418),
 ('использование', 0.5819722414016724),
 ('существенный', 0.579962432384491),
 ('достигаться', 0.5796698927879333),
 ('достигаемый', 0.5791462659835815)]

In [117]:
def most_probable(wv, words, n=4, enable_logging=True):
    if len(words) <= n:
        return words

    wv.init_sims()
    
    used_words = set([word for word in words if word in wv])
    if len(used_words) != len(words):
        ignored_words = set(words) - set(used_words)
        if enable_logging:
            logger.warning("vectors for words %s are not present in the model, ignoring these words", ignored_words)
    if not used_words:
        logger.warning('not in vocabulary %s', words)
        return []
    vectors = np.vstack(wv.word_vec(word, use_norm=True) for word in used_words).astype(np.float32)
    mean = gensim.matutils.unitvec(vectors.mean(axis=0)).astype(np.float32)
    dists = np.dot(vectors, mean)
    most = sorted(zip(dists, used_words), reverse=True)[:n]
    
    return [word for sim, word in most]

In [8]:
sent = 'РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['описывать', 'получение', 'формула']


In [50]:
sent = 'технический результат полезный модель заключаться'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['заключаться', 'результат', 'технический']


In [10]:
sent = 'новый технологичный способ получение'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['способ', 'получение', 'новый']


#### Теперь придумаем новую фразу

In [11]:
sent = 'новизна данный изобретение в том что гидроусилитель статический'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'в'} are not present in the model, ignoring these words


final n-gram: ['данный', 'новизна', 'изобретение']


Здесь важно натренировать w2v так чтобы в корпусе присутствовали все слова кроме стоп слов, например, если убрать "данный", то получаем:

In [12]:
sent = 'новизна изобретение в том что гидроусилитель статический '
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'в'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'изобретение', 'статический']


А это уже хуже. Поэтому надо брать достаточное скользящее окно, например, 5 и в нем выбирать 3 наиболее вероятных терма. Также плохо брать слишком большое окно

In [13]:
sent = 'новизна изобретение в том что гидроусилитель статический с повышенный уровень отказоустойчивость'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'с', 'в'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'отказоустойчивость', 'уровень']


# Реализация

In [27]:
def iconv(sent, n=3):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(sent)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [78]:
def convolv_on_doc(doc, window=5, n=3):
    """
    doc - list object with sentences as lists of words
    window - windows size
    n - n-gram size
    """
    for sent in doc:
        step = max(n, min(window, len(sent)))
        for ngram in iconv(sent, step):
            yield ngram

**Explanation**

In [79]:
doc = [[1,2],[3,4,5],[6,7,8,9],[10,11,12,13,14],[15,16,17,18,19,20]]
for tup in convolv_on_doc(doc):
    print(tup)

(3, 4, 5)
(6, 7, 8, 9)
(10, 11, 12, 13, 14)
(15, 16, 17, 18, 19)
(16, 17, 18, 19, 20)


In [82]:
from nltk.corpus import stopwords

stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list = set(stop_list)


fn = '../data/validate/_ab_68_2006139128A10010101RU.txt'
with open(fn, 'r') as f:
    doc_text = f.read()

sents = sent_tokenize(doc_text)

doc = [tokenize(s, stop_list) for s in sents]

In [118]:
def get_convolved_ngrams(doc, window=5, n=3):
    result = []
    for ngram in convolv_on_doc(doc, window, n):
        probable_ngram = most_probable(wv, ngram, n, enable_logging=False)
        if len(probable_ngram):
            avg_ngram = np.mean(wv[probable_ngram], axis=0)
            result.append((probable_ngram, avg_ngram))
    return result

In [119]:
convolved_doc = get_convolved_ngrams(doc, window=5, n=3)

not in vocabulary ('crc', 'handbook', 'of', 'chemistry', 'and')
not in vocabulary ('handbook', 'of', 'chemistry', 'and', 'physics')
not in vocabulary ('of', 'chemistry', 'and', 'physics', 'edition')
not in vocabulary ('cf', 'ch', 'chf', 'hfc', 'ze')
not in vocabulary ('cf', 'cf', 'ch', 'chf', 'hfc')
not in vocabulary ('cf', 'ch', 'chf', 'hfc', 'mce')
not in vocabulary ('cf', 'ch', 'chc', 'ch', 'chc')
not in vocabulary ('cf', 'cf', 'ch', 'chc', 'ch')
not in vocabulary ('cf', 'ch', 'chc', 'ch', 'chc')


In [101]:
len(convolved_doc)

1201

# Пример

In [123]:
inspect_result('_fc_bc_2010110681A10010101RU', 0,5)
# показательный пример того что наличие общих фраз еще ничего не гарантирует _fc_bc_2010110681A10010101RU

﻿способ предупреждение чрезвычайный ситуация на сибиреязвенный скотомогильник и территория .
 изобретение относиться к область защита потенциально опасный объект , население и территория субъект российский федерация и российский федерация в целый от чрезвычайный ситуация ( чс ) природный , техногенный и биолый - социальный характер , а также террористический акт , в частность от заражение население и территория возбудитель сибирский язва .
 технический результат ( полезность изобретение ) состоять в повышение эффективность проводить мероприятие по обеспечение целостность и сохранность сибиреязвенный скотомогильник , обнаружение и регистрация потенциально опасный биологический объект ( побо ) и опасный биологический объект ( обо ) по сибирский язва , располагать на территория российский федерация .
 предлагать способ предупреждение чс на сибиреязвенный скотомогильник и территория позволять : определять неблагополучный по сибирский язва территория ; систематизировать получать информация 

способ оповещение население и спецслужба о чрезвычайный ситуация и устройство сопряжение и мониторинг для о реализация  . 
 1 .
 способ оповещение население и спецслужба о чрезвычайный ситуация и устройство для о реализация , заключать в том , что непрерывно проводиться мониторинг стационарный и мобильный устройство на предмет возникновение чрезвычайный ситуация на все опасный производство , школа , больница , детский сад и место массовый скопление население в населенный пункт и передача информация по нескольких альтернативный канал связь , такой как телефонный сеть общий пользование , сотовый связь , радиосвязь , локальный вычислительный сеть , на центральный пост управление чрезвычайный ситуация населенный пункт , где производиться обработка весь поступать информация , прогнозирование развитие чрезвычайный ситуация , выработка алгоритм действие по устранение чрезвычайный ситуация и минимизация ущерб , с учет все фактор и в зависимость от уровень опасность возникать чрезвычайный ситуа

способ предупреждение чрезвычайный ситуация на сибиреязвенный скотомогильник и территория  . 
 1 .
 способ предупреждение чрезвычайный ситуация на сибиреязвенный скотомогильник и территория , состоять в периодический , не менее два раз в год ( весна и осень ) , проверка ветеринарно-санитарного состояние скотомогильник , восстановление оседать земляной насыпь , ограждение территория защитный периметр , отраншеивание и обвалцовывание внутри защитный периметр , оканавливание траншея заданный глубина , бетонирование траншея и территория скотомогильник в случай о подтопление , отличать тем , что оценка состояние скотомогильник и территория осуществлять непрерывно при помощь комплексный ( мобильный и стационарный ) система мониторинг .
 2 .
 способ по п.1 , отличать тем , что микробиологический контроль почва на территория скотомогильник и прилегать к он территория осуществлять с использование мобильный подсистема мониторинг ( микробиологический лаборатория ) .
 3 .
 способ по п.1 , отличать

устройство для экспериментальный отбор дезинфектант для обеззараживание грунт , заражать спор сибирский язва  . 
 полезный модель относиться к технический устройство для медицинский и ветеринарный дезинфектология , предназначать для экспериментальный отбор наиболее эффективный дезинфектант из большой число известный или новый дезинфицирующий средство для обеззараживание грунт в сибиреязвенный скотомогильник и на участок почва , контаминированный спор возбудитель сибирский язва или другой опасный микроорганизм , возникать в результат захоронение , забой , падеж больной животное .
 устройство состоять из закрытый герметичный камера , изготовлять из металл и стекло или оргстекло , размер 500×500×2500 мм , с герметичный люк с выпускной клапан с дверь для размещение грунт и биоагент , приемный пробоотборник и узел крепление приемный пробоотборник к колонна , опора колонна для установка в помещение или на открытый площадка , лестница для подъем и загрузка колонна грунт , а также для подъем и

способ определение загрязненность почвенный покров техногенный компонент  . 
 изобретение относиться к мониторинг окружающий среда для выделение участок загрязнение почвенный покров , установление источник загрязнение и зона влияние промышленный предприятие .
 согласно заявлять способ отбирать проба почва , они высушивать , просеивать до фракция менее 1 мм , просевать часть каждый проба анализировать при помощь бинокулярный стереоскопический микроскоп , затем устанавливать процентный соотношение техногенный компонент , по который проводить построение изолиния и по трехкратный превышение значение относительно фон выделять загрязненный предприятие участок почвенный покров .
 технический результат : экспрессный определение загрязненность почвенный покров .
 1 табл . , 3 ил .  . 
 способ определение загрязненность почвенный покров техногенный компонент , включать отбор проба почва , они высушивание , просеивание до фракция менее 1 мм , определение в они наличие загрязнять компонент , вынес

способ почвенный мониторинг земля  . 
 1 .
 способ почвенный мониторинг земля , включать они космический съемка , аэросъемка , картографирование , оценка качество и классификация они почва , расчет они стоимость , отличать тем , что на снимок земля обследовать территория визуально определять по фототон , структура изображение , спектральный яркость ареал почвенный разность , негативный свойство почва , выделять они условный обозначение , сканировать снимок и топографический основа земля обследовать территория , совмещать получать отображение на экран компьютер по опознавать точка в один общий двухслойный отображение , объектизировать , генерализировать , векторизовать на он граница земля с различный почвенный и геоморфологический признак , зафиксированный съемка , разделять земля на зона по вид они оптимальный использование , указывать условный обозначение граница этот зона , они конфигурация , площадь , тип , подтип , характеристика и качество почва , вид использование земля , при нео

способ прогнозирование фасциолез и устройство для о осуществление  . 
 способ прогнозирование фасциолез и устройство для о осуществление относиться к ветеринарный паразитология .
 изобретение обеспечивать повышение точность и достоверность прогнозирование .
 сущность способ заключаться в следующий : эпизоотологический и биоэкологический данные дополняться результат обследование постоянный и временной водоем на пастбище на наличие адолескария , и при обнаружение 10-15 экземпляр прогнозировать высокий риск фасциолез .
 для выявление адолескария использовать устройство в вид дисковый поплавок из бесцветный светопрозрачный физиологически инертный полистироть или ацетилцеллюлоза , окрашивать изнутри по равный сектор в белый , оранжевый , зеленый , голубой и черный цвет .
 достоверность способ - 76-100% , что подтверждать гельминтологический обследование и вскрытие животное опытный группа .
 способ простой в осуществление .  .   .   .
